In [3]:
import openai
import fitz  # PyMuPDF
from dotenv import load_dotenv
import os
import pandas as pd
from io import StringIO
#from prompt import prompt


In [ ]:
prompt = """
Eres un asistente especializado en estructurar información de facturas. Te proporcionaré texto sin formato extraído de diferentes facturas, y tu tarea es transformarlo en un CSV con punto y coma (;) como separador de campos.

📌 Requerimientos de extracción y formato:
1️⃣ fecha_factura: Extrae la fecha de emisión de la factura y conviértela al formato dd/mm/aaaa (día/mes/año). En el caso de que haya varias fechas elige la que sea fecha de emision o fecha de pedido.
2️⃣ proveedor: Extrae el nombre de la empresa emisora de la factura y conviértelo a minúsculas sin signos de puntuación (puede contener letras y números).
3️⃣ concepto: Extrae la descripción del producto o servicio facturado. Si hay varias descripciones, elige la más representativa.
4️⃣ importe: Extrae el monto total de la factura (considera que es la sumatoria de los montos individuales de todos los items de la factura) y conviértelo al formato español (usa la coma como separador decimal y elimina separadores de miles).
5️⃣ moneda: Determina la moneda de la factura:
   - Si contiene "EUR" o "€" o cualquier otro indicador de que la moneda son euros, devuelve "euros".
   - Si contiene "USD" o "$" o cualquier otro indicador de que la moneda son dólares US, devuelve "dolares".
   - Si la moneda no está clara, devuelve "otros".

📌 Formato de salida obligatorio:
✅ **Siempre incluye la siguiente cabecera como primera línea (sin excepción):**
fecha_factura;proveedor;concepto;importe;moneda
✅ Luego, en cada línea siguiente, proporciona únicamente los valores extraídos en ese mismo orden.
✅ No agregues encabezados repetidos en ninguna circunstancia.
✅ No generes líneas vacías.
✅ No incluyas explicaciones ni comentarios adicionales.

📌 **Ejemplo de salida esperada en CSV:**
fecha_factura;proveedor;concepto;importe;moneda
10/01/2024;openai llc;ChatGPT Plus Subscription;20,00;dolares
11/01/2024;amazon services europe sà r.l.;soporte de micrófono ajustable;19,99;euros
12/01/2024;raiola networks sl;hosting base ssd 20;119,91;euros

📌 **Instrucciones finales**:
- Devuelve solo el CSV limpio, sin repeticiones de encabezado ni líneas vacías.
- **Si no puedes extraer datos, responde exactamente con `"error"` sin comillas**.
"""

In [12]:
import openai
import fitz  # PyMuPDF
from dotenv import load_dotenv
import os
import pandas as pd
from io import StringIO
#from prompt import prompt

# Cargar variables de entorno desde el archivo .env
load_dotenv(".env")

# Obtener la clave de API de OpenAI desde las variables de entorno
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
cliente = openai.OpenAI(api_key=OPENAI_API_KEY)


def extraer_texto_pdf(ruta_pdf):

    doc = fitz.open(ruta_pdf)  # Abrir PDF
    text = "\n".join([page.get_text("text") for page in doc])  # Extraer texto
    return text


def estructurar_texto(texto):
    """Envía el texto a OpenAI y obtiene la respuesta estructurada en CSV,
    asegurando que solo devuelva datos válidos o 'error' en caso de problema."""


    respuesta = cliente.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "Eres un experto en extracción de datos de facturas. Devuelve solo el CSV sin explicaciones ni mensajes adicionales. Si no puedes extraer datos, devuelve exactamente la palabra 'error' sin comillas.",
            },
            {
                "role": "user",
                "content": prompt + "\n Este es el texto a parsear:\n" + texto,
            },
        ],
    )

    csv_respuesta = respuesta.choices[0].message.content.strip()
    return csv_respuesta


def csv_a_dataframe(csv):
    """Convierte el texto CSV en un DataFrame de pandas, asegurando que 'importe' sea numérico."""

    # Definir los tipos de datos para cada columna
    dtype_cols = {
        "fecha_factura": str,
        "proveedor": str,
        "concepto": str,
        "importe": str,  # Se leerá primero como str para poder limpiar comas
        "moneda": str,
    }

    # Leer el CSV en un DataFrame con los tipos especificados
    df_temp = pd.read_csv(StringIO(csv), delimiter=";", dtype=dtype_cols)

    # Convertir 'importe' a float, asegurando que los valores con coma se conviertan correctamente
    df_temp["importe"] = pd.to_numeric(
        df_temp["importe"].str.replace(",", "."), errors="coerce"
    )

    return df_temp


In [13]:
#import funciones
import pandas as pd
import os
from sqlalchemy import create_engine

# Crear un DataFrame vacío para almacenar todas las facturas
df = pd.DataFrame()

# Recorrer todas las carpetas dentro de la carpeta "facturas"
#for carpeta in sorted(os.listdir("./facturas")):
ruta_carpeta = os.path.join("/content/facturas/")

# Recorrer todos los archivos dentro de la carpeta
for archivo in os.listdir(ruta_carpeta):
    # Filtrar solo archivos PDF (ignorar carpetas y otros archivos)
    if archivo.lower().endswith('.pdf') and os.path.isfile(os.path.join(ruta_carpeta, archivo)):
        ruta_pdf = os.path.join(ruta_carpeta, archivo)

        print(f"📄 Procesando factura: {ruta_pdf}")

        # Extraer texto de la factura
        texto_no_estructurado = extraer_texto_pdf(ruta_pdf)
        print("noestructiurado")
        print(texto_no_estructurado)
        # Estructurar el texto de la factura
        texto_estructurado = estructurar_texto(texto_no_estructurado)
        print("estructiurado")
        print(texto_estructurado)
        # Convertir texto estructurado en dataframe
        df_factura = csv_a_dataframe(texto_estructurado)
        print("Dataframe:df_factura:")
        print(df_factura)

        # Anexar el dataframe de la factura al dataframe general
        df = pd.concat([df, df_factura], ignore_index=True)
        print("Dataframe:dftotal:")
        print(df)

 # Si la moneda es "dolares" convertir a euros multiplicando por 0,9243
 # df.loc[df["moneda"] == "dolares", "importe"] *= 0.9243


# Eliminar las columnas no esenciales
#df = df.iloc[:, 0:4]


📄 Procesando factura: /content/facturas/Factura_006.pdf
noestructiurado
HELADERÍA ARTESANAL "SABORES DEL SUR"
Bahía Blanca, Argentina
FACTURA B - CON IVA DISCRIMINADO
Número:
F-001-001-0000006
Fecha:
12/05/2024
Cliente:
EMPRESA "FRÍO POLAR" S.A.
CUIT:
33-98765432-1
IVA:
Responsable Inscripto
Producto
Cantidad
P. Unit.
IVA %
IVA $
Total
Helado Industrial 10 kg
2
$65.000
21%
$27.300
$157.300
Máquina Heladera Prof.
1
$45.000
21%
$9.450
$54.450
TOTAL CON IVA: $211.750
Página 1

estructiurado
fecha_factura;proveedor;concepto;importe;moneda
12/05/2024;heladería artesanal sabores del sur;total con iva;211,75;dolares
Dataframe:df_factura:
  fecha_factura                            proveedor       concepto  importe  \
0    12/05/2024  heladería artesanal sabores del sur  total con iva   211.75   

    moneda  
0  dolares  
Dataframe:dftotal:
  fecha_factura                            proveedor       concepto  importe  \
0    12/05/2024  heladería artesanal sabores del sur  total con iva   211.7

In [8]:
engine = create_engine("sqlite:///facturas.db")
# Guardar el DataFrame final en una bbdd sqlite, añadiendo los datos en lugar de reemplazarlos
df.to_sql("facturas", engine, if_exists="append", index=False)

# Cerrar la conexión a la base de datos
engine.dispose()

print("Proceso de extracción y estructuración de facturas completado exitosamente.")
print("Datos guardados en la base de datos 'facturas.db'.")


Proceso de extracción y estructuración de facturas completado exitosamente.
Datos guardados en la base de datos 'facturas.db'.


In [9]:
# ... tu código actual ...
import sqlite3


# 🔍 NUEVO CÓDIGO PARA VER LOS DATOS
print("\n" + "="*50)
print("📊 VISUALIZACIÓN DE DATOS EN SQLite")
print("="*50)

# Conectar a la base de datos
conn = sqlite3.connect("/content/facturas.db")

# 1. Ver todas las tablas en la base de datos
print("\n1. 📋 TABLAS EN LA BASE DE DATOS:")
tablas = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tablas)

# 2. Ver la estructura de la tabla 'facturas'
print("\n2. 🏗️ ESTRUCTURA DE LA TABLA 'facturas':")
estructura = pd.read_sql("PRAGMA table_info(facturas);", conn)
print(estructura)

# 3. Ver las primeras filas de datos
print("\n3. 📄 PRIMERAS 10 FILAS DE DATOS:")
datos = pd.read_sql("SELECT * FROM facturas LIMIT 19;", conn)
print(datos)

# 4. Ver estadísticas básicas
print("\n4. 📈 ESTADÍSTICAS BÁSICAS:")
print(f"Total de registros: {pd.read_sql('SELECT COUNT(*) as total FROM facturas;', conn)['total'][0]}")

# 5. Ver columnas disponibles
print("\n5. 🗂️ COLUMNAS DISPONIBLES:")
print(df.columns.tolist())

# 6. Ver tipos de datos
print("\n6. 🔧 TIPOS DE DATOS:")
print(df.dtypes)

# Cerrar conexión
conn.close()

print("\n✅ Visualización completada!")


📊 VISUALIZACIÓN DE DATOS EN SQLite

1. 📋 TABLAS EN LA BASE DE DATOS:
       name
0  facturas

2. 🏗️ ESTRUCTURA DE LA TABLA 'facturas':
   cid           name   type  notnull dflt_value  pk
0    0  fecha_factura   TEXT        0       None   0
1    1      proveedor   TEXT        0       None   0
2    2       concepto   TEXT        0       None   0
3    3        importe  FLOAT        0       None   0
4    4         moneda   TEXT        0       None   0

3. 📄 PRIMERAS 10 FILAS DE DATOS:
  fecha_factura                            proveedor  \
0    12/05/2024  heladería artesanal sabores del sur   
1    05/04/2024  heladería artesanal sabores del sur   
2    22/03/2024  heladería artesanal sabores del sur   
3    18/04/2024  heladería artesanal sabores del sur   
4    02/05/2024  heladería artesanal sabores del sur   
5    15/03/2024  heladería artesanal sabores del sur   

                        concepto    importe   moneda  
0                  total con iva  211750.00  dolares  
1  helado